Скрипт выгружает заголовки и описание роликов.

Последнее обновление: 2021-12-26

## Библиотеки и настройки ##

In [1]:
#Загружаю нужные библиотеки

import pandas as pd

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# путь к файлам

file_path = 'C:/00_Data/'
proxy_list_path = 'C:/work/proxy_list.txt'

In [5]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]

In [6]:
print('Рабочих прокси — ' + str(len(proxy_list)))

Рабочих прокси — 8


In [7]:
# номер прокси проще прописать вручную, ввод со строки постоянно глючит

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

#proxy_number = 1

Номер прокси, от 0 до 7
4


In [8]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [9]:
# логин - пароль

print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

Login: borisgimatij
Pass: IqJ9Na7Vug


In [10]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки на видео ##

In [11]:
# адрес канала

url = 'https://www.youtube.com/channel/UCQ4YOFsXjG9eXWZ6uLj2t2A/videos'

In [12]:
# открываю ссылку

driver.get(url)

In [20]:
# прокручиваю страницу вниз

screen_height = 1080
video_number = 0

while video_number != (
    len(driver.find_elements_by_class_name('style-scope.ytd-grid-video-renderer'))):
    
    video_number = len(driver.find_elements_by_class_name('style-scope.ytd-grid-video-renderer'))
    
    # 
    driver.execute_script(f"window.scrollTo(0, {screen_height})")
    sleep(2)
    screen_height += 1080 

In [22]:
for i in tqdm(range(10)):
    
    driver.execute_script(f"window.scrollTo(0, {screen_height})")
    sleep(2)
    screen_height += 1080

  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
data = [] # список для данных

# блоки с видео
video_blocks = driver.find_elements_by_css_selector('h3.style-scope.ytd-grid-video-renderer')

for item in tqdm(video_blocks):
    
    # заголовок видео
    video_title = item.find_element_by_tag_name('a').text
    
    # ссылка
    video_page = item.find_element_by_tag_name('a').get_attribute('href')   
    
    data.append([video_title, video_page])

In [27]:
# Записываю в датафрейм

head_list = ['video_title', 'video_page']

work_links = pd.DataFrame(data, columns=head_list)

In [28]:
work_links.sample(3)

,video_title,video_page
2063,Соловьёв LIVE / 7 мая 2020 года,https://www.youtube.com/watch?v=GLFn7o9zu-Y
2139,Что вы делаете на самоизоляции? / Грушевский и...,https://www.youtube.com/watch?v=MenHmYRn2xY
497,Американские СМИ УНИЧТОЖИЛИ Байдена за трагеди...,https://www.youtube.com/watch?v=EcEDoMk4Hn0


In [29]:
work_links.shape

(2170, 2)

In [49]:
(driver
 .find_element_by_class_name('style-scope.ytd-video-secondary-info-renderer')
 .find_element_by_id('more')
).click()

In [51]:
# описание

driver.find_element_by_class_name('content.style-scope.ytd-video-secondary-info-renderer').text

'Следующая неделя для россиян станет нерабочей с сохранением зарплаты. Об этом объявил Владимир Путин в обращении к россиянам в связи с распространением коронавируса. Президент также заявил, что всенародное голосование по поправкам в Конституцию переносится на более поздний срок. Глава государства отметил, что приоритет – здоровье граждан. Что будет дальше? Обсудим с экспертами в «Соловьёв LIVE»'

In [30]:
text = driver.page_source

In [69]:
need_text = re.findall(r'{"@context":"https://schema.org","@type":"VideoObject".+}', text)

In [79]:
type(need_text)

list

In [85]:
description = re.sub(r'.+description":"', '', need_text[0])

In [87]:
description = re.sub(r'".+', '', description)

In [113]:
re.findall(r'uploadDate":".+"', need_text[0])

['uploadDate":"2020-03-25","genre":"News & Politics","author":"Соловьёв LIVE","publication":[{"@type":"BroadcastEvent","isLiveBroadcast":true,"startDate":"2020-03-25T18:29:56+00:00","endDate":"2020-03-25T19:58:34+00:00"']

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
item.find_element_by_tag_name('a').get_attribute('href')

In [ ]:
data[:-5]

In [ ]:
data[-5:]

In [33]:
work_links.to_excel(file_path + 'pomet_urls.xlsx', encoding='utf8', index=False)